## Train model
Danh sách class: <br>
    0. Correct <br>
    1. Chân quá hẹp <br >
    2. Chân quá rộng <br >
    3. Khoảng cách giữa 2 đầu gối quá nhỏ <br >
    4. Xuống quá xâu <br >
    5. Lỗi gập gập lưng <br >

In [16]:
import mediapipe as mp
import cv2
import pandas as pd
import pickle
import numpy as np
import csv
import seaborn as sns
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

# Drawing helpers
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

# 1. Khởi tạo keypoints và các phương thức 
Các keypoints cần lấy :
    "NOSE",
    "LEFT_SHOULDER",
    "RIGHT_SHOULDER",
    "LEFT_HIP",
    "RIGHT_HIP",
    "LEFT_KNEE",
    "RIGHT_KNEE",
    "LEFT_ANKLE",
    "RIGHT_ANKLE",

Dataframe gồm label và các features là toạ độ keypoints gồm 4 thành phần: x, y, z, visibility
- x, y: tọa độ x, y của keypoints đã được chuẩn hóa min max scale
- z: độ sâu của điểm ảnh theo điểm giữa hông 
- visibility: biểu thị khả năng có thể nhìn thấy hoặc che khuất
 

In [17]:
#Chọn các điểm cần lấy
Important_kp = [
    "NOSE",
    "LEFT_SHOULDER",
    "RIGHT_SHOULDER",
    "LEFT_HIP",
    "RIGHT_HIP",
    "LEFT_KNEE",
    "RIGHT_KNEE",
    "LEFT_ANKLE",
    "RIGHT_ANKLE",
]

#Tạo header cho dataframe
header = ["label"]

for kp in Important_kp:
    header.extend([f"{kp}_x", f"{kp}_y", f"{kp}_z", f"{kp}_visibility"])

header

['label',
 'NOSE_x',
 'NOSE_y',
 'NOSE_z',
 'NOSE_visibility',
 'LEFT_SHOULDER_x',
 'LEFT_SHOULDER_y',
 'LEFT_SHOULDER_z',
 'LEFT_SHOULDER_visibility',
 'RIGHT_SHOULDER_x',
 'RIGHT_SHOULDER_y',
 'RIGHT_SHOULDER_z',
 'RIGHT_SHOULDER_visibility',
 'LEFT_HIP_x',
 'LEFT_HIP_y',
 'LEFT_HIP_z',
 'LEFT_HIP_visibility',
 'RIGHT_HIP_x',
 'RIGHT_HIP_y',
 'RIGHT_HIP_z',
 'RIGHT_HIP_visibility',
 'LEFT_KNEE_x',
 'LEFT_KNEE_y',
 'LEFT_KNEE_z',
 'LEFT_KNEE_visibility',
 'RIGHT_KNEE_x',
 'RIGHT_KNEE_y',
 'RIGHT_KNEE_z',
 'RIGHT_KNEE_visibility',
 'LEFT_ANKLE_x',
 'LEFT_ANKLE_y',
 'LEFT_ANKLE_z',
 'LEFT_ANKLE_visibility',
 'RIGHT_ANKLE_x',
 'RIGHT_ANKLE_y',
 'RIGHT_ANKLE_z',
 'RIGHT_ANKLE_visibility']

In [18]:
from math import sqrt

# rescale frame 1/2
def rescale_frame(frame, percent=50):
    '''
    Rescale a frame to a certain percentage compare to its original frame
    '''
    width = int(frame.shape[1] * percent/ 100)
    height = int(frame.shape[0] * percent/ 100)
    dim = (width, height)
    return cv2.resize(frame, dim, interpolation =cv2.INTER_AREA)

#Tính khoảng cách giữa 2 điểm trong không gian 2D
def calculate_distance(pointX, pointY) -> float:
    '''
    Calculate a distance between 2 points
    '''

    x1, y1 = pointX
    x2, y2 = pointY

    return sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)

#Tính góc giữa 3 điểm trong không gian 3D
def calculate_angle(pointA, pointB, pointC) -> float:
    """
    Tính góc giữa ba điểm trong không gian 3D (đơn vị: độ)
    a, b, c là danh sách chứa tọa độ 3D: [x, y, z]
    """

    #Độ sâu ảnh
    t = 33;
    A = np.array([pointA[0], pointA[1], pointA[2]/2])  # (xA, yA, zA)
    B = np.array([pointB[0], pointB[1], pointB[2]/2 ])  # (xB, yB, zB)
    C = np.array([pointC[0], pointC[1], pointC[2]/2])  # (xC, yC, zC)

    # Vector BA và BC
    BA = A - B
    BC = C - B

    # Tính tích vô hướng
    dot_product = np.dot(BA, BC)
    
    # Tính độ dài vector
    norm_BA = np.linalg.norm(BA)
    norm_BC = np.linalg.norm(BC)

    # Tính góc bằng công thức cos(theta) = (A.B) / (|A| * |B|)
    cos_theta = dot_product / (norm_BA * norm_BC)
    
    # Chuyển từ radian sang độ
    angle = np.degrees(np.arccos(np.clip(cos_theta, -1.0, 1.0)))

    return angle

#Đọc dữ liệu từ frame trả về points
def extract_important_keypoints(results) -> list:
    '''
    Extract important keypoints from mediapipe pose detection
    '''
    landmarks = results.pose_landmarks.landmark

    data = []
    for lm in Important_kp:
        keypoint = landmarks[mp_pose.PoseLandmark[lm].value]
        data.append([keypoint.x, keypoint.y, keypoint.z, keypoint.visibility])
    
    return np.array(data).flatten().tolist()


In [19]:
#Lỗi 1 - 2: ratio = feet/shoulder
Min_radio_feet_shoulder = 0.933233
Max_radio_feet_shoulder = 1.4

#Lỗi 3 : ratio = knee/feet
Min_up_radio_knee_feet = 0.56 
Min_middle_ratio_knee_feet = 0.66 
Min_down_ratio_knee_feet = 1 

#Lỗi 4: knee angle < 50
Min_knee_angle = 50

#Lỗi 5: ratio = knee_angle/hip_angle
Max_ratio_knee_hip = 0.98


In [28]:
df = pd.DataFrame(columns=header)
df_test = pd.DataFrame(columns=header)
df_va = pd.DataFrame(columns=header)
df 

,label,NOSE_x,NOSE_y,NOSE_z,NOSE_visibility,LEFT_SHOULDER_x,LEFT_SHOULDER_y,LEFT_SHOULDER_z,LEFT_SHOULDER_visibility,RIGHT_SHOULDER_x,...,RIGHT_KNEE_z,RIGHT_KNEE_visibility,LEFT_ANKLE_x,LEFT_ANKLE_y,LEFT_ANKLE_z,LEFT_ANKLE_visibility,RIGHT_ANKLE_x,RIGHT_ANKLE_y,RIGHT_ANKLE_z,RIGHT_ANKLE_visibility


# 2. Hàm duyệt data lỗi chân quá hẹp

In [29]:
#Xử lý từng video để tính khoảng cách chân, vai
def detection_1_error(Video_folder, Video_name, type):
    
    Cap = cv2.VideoCapture(f"{Video_folder}/{Video_name}")

    images = []  # Danh sách lưu hình ảnh
    max_images = 10  # Số hình ảnh tối đa hiển thị cùng lúc

    with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
        while Cap.isOpened():
            ret, image = Cap.read()

            if not ret:
                break

            # Chuyển ảnh sang RGB
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            image = rescale_frame(image, percent=50)
            image.flags.writeable = True  # Make the image writable

            #trích xuất kpkp
            results = pose.process(image)

            # Kiểm tra có nhận được keypoint không
            if not results.pose_landmarks:
                continue
            
            #khởi tạo biến tính khoảng cách
            shoulder_width = feet_width = None
            
            try:
                # Lấy kp từ frame
                # row = extract_important_keypoints(results)
                # X = pd.DataFrame([row], columns=header[1:])

                # Tinhs toán và so sánh khoảng cách vai, hông, chân
                landmarks = results.pose_landmarks.landmark
                # Khoảng cách giữa 2 vai
                left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
                right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]

                shoulder_width = calculate_distance(left_shoulder, right_shoulder)

                # Khoảng cách giữa 2 chân
                left_foot_index = [landmarks[mp_pose.PoseLandmark.LEFT_FOOT_INDEX.value].x, landmarks[mp_pose.PoseLandmark.LEFT_FOOT_INDEX.value].y]
                right_foot_index = [landmarks[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX.value].y]

                feet_width = calculate_distance(left_foot_index, right_foot_index)

                # Kiểm tra lỗi
                row = []
                if(feet_width/shoulder_width < Min_radio_feet_shoulder):
                    row.append(1)
                    for lm in Important_kp:
                        keypoint = landmarks[mp_pose.PoseLandmark[lm].value]
                        row.extend([keypoint.x, keypoint.y, keypoint.z, keypoint.visibility])

                if len(row) == len(df.columns):
                    if type == "train":
                        df.loc[len(df)] = row
                    elif type == "test":
                        df_test.loc[len(df_test)] = row
                    elif type == "validate":
                        df_va.loc[len(df_va)] = row
                else:
                    print(f"Lỗi! row có {len(row)} phần tử nhưng df có {len(df.columns)} cột.")

                # Nền
                cv2.rectangle(image, (0, 0), (500, 60), (245, 117, 16), -1)

                # Display feet distance
                cv2.putText(image, "FEET", (15, 12), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
                cv2.putText(image, str(round(feet_width, 2)), (10, 40), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

                # Display shoulder distance
                cv2.putText(image, "SHOULDER", (95, 12), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
                cv2.putText(image, str(round(shoulder_width, 2)), (90, 40), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

                # Display label
                cv2.putText(image, "LABEL", (200, 12), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
                cv2.putText(image, str(round(row[0], 2)), (200, 40), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

            except Exception as e:
                print(f"Error: {e}")

            # Draw landmarks and connections
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS, mp_drawing.DrawingSpec(color=(244, 117, 66), thickness=2, circle_radius=4), mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2))

            images.append(image)

            # Khi đủ 10 ảnh, hiển thị tất cả cùng lúc
            # if len(images) == max_images:
            #     fig, axes = plt.subplots(5, 2, figsize=(32, 18))  # Tạo grid 2x5
            #     for i, ax in enumerate(axes.flat):
            #         ax.imshow(images[i])
            #         ax.set_aspect('auto')
            #         ax.axis("off")  # Ẩn trục tọa độ
            #     plt.show()
            #     images = []  # Reset danh sách sau khi hiển thị

        Cap.release()

In [ ]:
import os

video_folder = "Data/train/Chan_qua_hep"
video_files = [f for f in os.listdir(video_folder) if f.endswith(".mp4")]

for video_file in video_files:
    detection_1_error(video_folder, video_file, "train")

In [22]:
df 

,label,NOSE_x,NOSE_y,NOSE_z,NOSE_visibility,LEFT_SHOULDER_x,LEFT_SHOULDER_y,LEFT_SHOULDER_z,LEFT_SHOULDER_visibility,RIGHT_SHOULDER_x,...,RIGHT_KNEE_z,RIGHT_KNEE_visibility,LEFT_ANKLE_x,LEFT_ANKLE_y,LEFT_ANKLE_z,LEFT_ANKLE_visibility,RIGHT_ANKLE_x,RIGHT_ANKLE_y,RIGHT_ANKLE_z,RIGHT_ANKLE_visibility
0,1.0,0.536996,0.249936,-0.301145,0.999949,0.571622,0.338894,-0.078298,0.999725,0.481677,...,-0.067078,0.994322,0.558694,0.906207,0.092379,0.973042,0.512528,0.914821,0.083676,0.987352
1,1.0,0.536921,0.255405,-0.301153,0.999947,0.571646,0.340262,-0.077996,0.999735,0.481808,...,-0.074440,0.994011,0.558658,0.906270,0.089135,0.971412,0.512588,0.914797,0.076281,0.986404
2,1.0,0.537403,0.272339,-0.306074,0.999947,0.571647,0.348699,-0.083166,0.999750,0.481798,...,-0.107100,0.993944,0.558685,0.906296,0.059279,0.970751,0.512612,0.914797,0.038212,0.985989
3,1.0,0.537598,0.282832,-0.300753,0.999947,0.571760,0.361542,-0.081161,0.999765,0.481674,...,-0.094014,0.993858,0.559341,0.906479,0.050163,0.970697,0.512875,0.914826,0.047682,0.985630
4,1.0,0.538748,0.296727,-0.300915,0.999949,0.571948,0.371516,-0.081955,0.999783,0.481692,...,-0.118264,0.993921,0.560000,0.906567,0.042005,0.970954,0.512974,0.915044,0.034253,0.985670
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1360,1.0,0.502052,0.164798,-0.200236,0.999942,0.555950,0.271383,-0.066368,0.999930,0.462536,...,0.026470,0.987777,0.514152,0.882497,0.113064,0.981108,0.476988,0.877122,0.165850,0.985852
1361,1.0,0.502024,0.164827,-0.197816,0.999942,0.555996,0.271385,-0.066375,0.999930,0.462649,...,0.028332,0.987655,0.514392,0.884180,0.110009,0.981414,0.476964,0.877098,0.166238,0.985932
1362,1.0,0.502019,0.164854,-0.198327,0.999941,0.556103,0.271424,-0.067913,0.999929,0.462671,...,0.028563,0.987513,0.514563,0.883134,0.114635,0.981558,0.477222,0.875442,0.167355,0.985941
1363,1.0,0.502188,0.164731,-0.202035,0.999941,0.556240,0.271480,-0.069484,0.999928,0.462673,...,0.028522,0.987413,0.514693,0.881331,0.117688,0.981581,0.477643,0.874598,0.169866,0.985906


# 2. Lỗi chân rộng


In [24]:
#Xử lý từng video để tính khoảng cách chân, vai
def detection_1_error(Video_folder, Video_name, type):
    
    Cap = cv2.VideoCapture(f"{Video_folder}/{Video_name}")

    images = []  # Danh sách lưu hình ảnh
    max_images = 10  # Số hình ảnh tối đa hiển thị cùng lúc

    with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
        while Cap.isOpened():
            ret, image = Cap.read()

            if not ret:
                break

            # Chuyển ảnh sang RGB
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            image = rescale_frame(image, percent=50)
            image.flags.writeable = True  # Make the image writable

            #trích xuất kpkp
            results = pose.process(image)

            # Kiểm tra có nhận được keypoint không
            if not results.pose_landmarks:
                continue
            
            #khởi tạo biến tính khoảng cách
            shoulder_width = feet_width = None
            
            try:
                # Lấy kp từ frame
                # row = extract_important_keypoints(results)
                # X = pd.DataFrame([row], columns=header[1:])

                # Tinhs toán và so sánh khoảng cách vai, hông, chân
                landmarks = results.pose_landmarks.landmark
                # Khoảng cách giữa 2 vai
                left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
                right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]

                shoulder_width = calculate_distance(left_shoulder, right_shoulder)

                # Khoảng cách giữa 2 chân
                left_foot_index = [landmarks[mp_pose.PoseLandmark.LEFT_FOOT_INDEX.value].x, landmarks[mp_pose.PoseLandmark.LEFT_FOOT_INDEX.value].y]
                right_foot_index = [landmarks[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX.value].y]

                feet_width = calculate_distance(left_foot_index, right_foot_index)

                # Kiểm tra lỗi
                row = []
                if(feet_width/shoulder_width > Max_radio_feet_shoulder):
                    row.append(2)
                    for lm in Important_kp:
                        keypoint = landmarks[mp_pose.PoseLandmark[lm].value]
                        row.extend([keypoint.x, keypoint.y, keypoint.z, keypoint.visibility])

                if len(row) == len(df.columns):
                    if type == "train":
                        df.loc[len(df)] = row
                    elif type == "test":
                        df_test.loc[len(df_test)] = row
                    elif type == "validate":
                        df_va.loc[len(df_va)] = row
                else:
                    print(f"Lỗi! row có {len(row)} phần tử nhưng df có {len(df.columns)} cột.")

                # Nền
                cv2.rectangle(image, (0, 0), (500, 60), (245, 117, 16), -1)

                # Display feet distance
                cv2.putText(image, "FEET", (15, 12), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
                cv2.putText(image, str(round(feet_width, 2)), (10, 40), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

                # Display shoulder distance
                cv2.putText(image, "SHOULDER", (95, 12), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
                cv2.putText(image, str(round(shoulder_width, 2)), (90, 40), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

                # Display label
                cv2.putText(image, "LABEL", (200, 12), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
                cv2.putText(image, str(round(row[0], 2)), (200, 40), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

            except Exception as e:
                print(f"Error: {e}")

            # Draw landmarks and connections
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS, mp_drawing.DrawingSpec(color=(244, 117, 66), thickness=2, circle_radius=4), mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2))

            images.append(image)

            # Khi đủ 10 ảnh, hiển thị tất cả cùng lúc
            # if len(images) == max_images:
            #     fig, axes = plt.subplots(5, 2, figsize=(32, 18))  # Tạo grid 2x5
            #     for i, ax in enumerate(axes.flat):
            #         ax.imshow(images[i])
            #         ax.set_aspect('auto')
            #         ax.axis("off")  # Ẩn trục tọa độ
            #     plt.show()
            #     images = []  # Reset danh sách sau khi hiển thị

        Cap.release()

In [25]:
import os

video_folder = "Data/train/Chan_qua_rong"
video_files = [f for f in os.listdir(video_folder) if f.endswith(".mp4")]

for video_file in video_files:
    detection_1_error(video_folder, video_file, "train")

In [26]:
df

,label,NOSE_x,NOSE_y,NOSE_z,NOSE_visibility,LEFT_SHOULDER_x,LEFT_SHOULDER_y,LEFT_SHOULDER_z,LEFT_SHOULDER_visibility,RIGHT_SHOULDER_x,...,RIGHT_KNEE_z,RIGHT_KNEE_visibility,LEFT_ANKLE_x,LEFT_ANKLE_y,LEFT_ANKLE_z,LEFT_ANKLE_visibility,RIGHT_ANKLE_x,RIGHT_ANKLE_y,RIGHT_ANKLE_z,RIGHT_ANKLE_visibility
0,1.0,0.536996,0.249936,-0.301145,0.999949,0.571622,0.338894,-0.078298,0.999725,0.481677,...,-0.067078,0.994322,0.558694,0.906207,0.092379,0.973042,0.512528,0.914821,0.083676,0.987352
1,1.0,0.536921,0.255405,-0.301153,0.999947,0.571646,0.340262,-0.077996,0.999735,0.481808,...,-0.074440,0.994011,0.558658,0.906270,0.089135,0.971412,0.512588,0.914797,0.076281,0.986404
2,1.0,0.537403,0.272339,-0.306074,0.999947,0.571647,0.348699,-0.083166,0.999750,0.481798,...,-0.107100,0.993944,0.558685,0.906296,0.059279,0.970751,0.512612,0.914797,0.038212,0.985989
3,1.0,0.537598,0.282832,-0.300753,0.999947,0.571760,0.361542,-0.081161,0.999765,0.481674,...,-0.094014,0.993858,0.559341,0.906479,0.050163,0.970697,0.512875,0.914826,0.047682,0.985630
4,1.0,0.538748,0.296727,-0.300915,0.999949,0.571948,0.371516,-0.081955,0.999783,0.481692,...,-0.118264,0.993921,0.560000,0.906567,0.042005,0.970954,0.512974,0.915044,0.034253,0.985670
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2857,2.0,0.515885,0.135050,-0.229636,0.999994,0.561906,0.250951,-0.025717,0.999979,0.467342,...,-0.006169,0.997307,0.582271,0.885328,0.187713,0.994357,0.453820,0.890234,0.174316,0.997102
2858,2.0,0.515551,0.135061,-0.229236,0.999994,0.561803,0.250958,-0.027120,0.999980,0.467330,...,-0.005158,0.997308,0.582204,0.885200,0.186860,0.994346,0.453821,0.889975,0.176889,0.997099
2859,2.0,0.515401,0.135066,-0.221628,0.999994,0.561670,0.250983,-0.024165,0.999980,0.467332,...,-0.005144,0.997310,0.582151,0.885000,0.184367,0.994354,0.453825,0.889823,0.176811,0.997092
2860,2.0,0.515334,0.135069,-0.214825,0.999994,0.561577,0.251021,-0.019305,0.999981,0.467340,...,-0.005611,0.997310,0.582117,0.884941,0.183298,0.994332,0.453815,0.889763,0.175257,0.997080


# 3. Lỗi gối hẹp
